In [1]:
import os
import pprint
import csv
import requests
import pandas as pd

releases: RELEASE-2023 RELEASE-2022 RELEASE-2021

all of this could be obtained from a few calls to the API, especially the site endpoint

https://data.neonscience.org/api/v0/releases/RELEASE-2023/data/package/DP1.10107.001/GUAN/2019-09?package=expanded
  obtains a zip archive

https://data.neonscience.org/api/v0/data/package/DP1.10107.001/GUAN/2019-09?package=expanded&release=RELEASE-2023
  obtains a zip archive

https://data.neonscience.org/api/v0/releases/RELEASE-2023/data/DP1.10107.001/GUAN/2019-09?package=expanded
  obtains a JSON object with links to files

https://data.neonscience.org/api/v0/releases/RELEASE-2023/data/DP1.10107.001/GUAN/2019-09/NEON.D04.GUAN.DP1.10107.001.mms_rawDataFiles.2019-09.expanded.20230113T225344Z.csv?package=expanded
  obtains a CSV file,
  but you have to know the exact name of the file

{'EML',
 'categoricalCodes',
 'mms_metagenomeDnaExtraction',
 'mms_metagenomeSequencing',
 'mms_rawDataFiles',
 'readme',
 'validation',
 'variables'}



In [2]:
useful_file_categories = [
 'mms_metagenomeDnaExtraction',
 'mms_metagenomeSequencing',
 'mms_rawDataFiles',
]

In [3]:
# neon_dl_dir = "/Users/MAM/Downloads/NEON_seq-metagenomic-microbe-soil"
# neon_dl_dir_tokens = neon_dl_dir.split("/")
# neon_basename = neon_dl_dir_tokens[-1]
# output_file = f"../local/{neon_basename}.tsv"


In [4]:
def write_dicts_to_tsv(data, filename):
    fieldnames = data[0].keys()  # Assume all dictionaries have the same keys

    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter='\t')
        writer.writeheader()
        writer.writerows(data)

In [5]:

# subdirectories = [name for name in os.listdir(neon_dl_dir) if os.path.isdir(os.path.join(neon_dl_dir, name))]
#
# len_list = []
# token_dict_list = []
# for i in subdirectories:
#     tokens = i.split(".")
#     len_list.append(len(tokens))
#     temp = {
#         "data_owner": tokens[0],
#         "geographical_domain": tokens[1],
#         "siteCode": tokens[2],
#         "productCode": f"{tokens[3]}.{tokens[4]}.{tokens[5]}",
#         "year-month": tokens[6],
#         "package": tokens[7],
#         "timestamp": tokens[8],
#         "releaseTag": tokens[9],
#     }
#     token_dict_list.append(temp)


In [6]:
# write_dicts_to_tsv(token_dict_list, output_file)

In [7]:
# first_token_dict = token_dict_list[0]
# pprint.pprint(first_token_dict)


In [8]:
# product_code = first_token_dict['productCode']
# release_tag = first_token_dict['releaseTag']

product_code = "DP1.10107.001"
release_tag = "RELEASE-2023"

url = f"https://data.neonscience.org/api/v0/products/{product_code}?release={release_tag}"
response = requests.get(url).json()
site_wise = response['data']['siteCodes']

In [9]:
lod = []
file_categories = set()
df_dict = {}

for i in site_wise:
    # print(i['siteCode'])
    availableDataUrls = i['availableDataUrls']
    for j in availableDataUrls:
        print(j)
        data_files_response = requests.get(j).json()
        data_file_objects = data_files_response['data']['files']
        for k in data_file_objects:
            data_file_name = k['name']
            data_file_name_tokens = data_file_name.split(".")
            data_file_name_dict = {
                "data_owner": data_file_name_tokens[0],
                "geographical_domain": data_file_name_tokens[1],
                "siteCode": data_file_name_tokens[2],
                "productCode": f"{data_file_name_tokens[3]}.{data_file_name_tokens[4]}.{data_file_name_tokens[5]}",
                "file_category": data_file_name_tokens[6],
                # "timestamp": data_file_name_tokens[7],
                # "extension": data_file_name_tokens[8],
            }
            if data_file_name_dict['file_category'] in useful_file_categories:
                # print(k['url'])
                if data_file_name_dict['file_category'] in df_dict:
                    more = pd.read_csv(k['url'])
                    df_dict[data_file_name_dict['file_category']] = pd.concat(
                        [df_dict[data_file_name_dict['file_category']], more], ignore_index=True)
                else:
                    df_dict[data_file_name_dict['file_category']] = pd.read_csv(k['url'])

for k, v in df_dict.items():
    print(k)
    v.to_csv(f"../local/{k}.csv", index=False)
    # print(v)


https://data.neonscience.org/api/v0/data/DP1.10107.001/ABBY/2017-06?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BARR/2017-08?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BART/2014-05?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BART/2014-06?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BART/2014-07?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BART/2014-08?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BART/2016-07?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BART/2017-06?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BLAN/2017-07?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BONA/2018-09?release=RELEASE-2023
https://data.neonscience.org/api/v0/data/DP1.10107.001/BONA/2019-08?release=RELEASE-2023
https://data.neonscie

In [ ]:
# write_dicts_to_tsv(lod, "../local/soil_metagenome_filename_tokens.tsv")

In [ ]:
# pprint.pprint(file_categories)